In [201]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from sklearn.metrics import accuracy_score
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexingTables = Elasticsearch()

In [2]:
readFile = pd.read_csv('articlesDataset/trainDatasetTitle', delimiter=',', header=None)
trainDataset = readFile.iloc[:,:].values

In [3]:
readFile = pd.read_csv('articlesDataset/validationDatasetTitle', delimiter=',', header=None)
validationDataset = readFile.iloc[:,:].values

In [193]:
readFile = pd.read_csv('articlesDataset/testDatasetTitle', delimiter=',', header=None)
testDataset = readFile.iloc[:,:].values

In [284]:
def getAccuracy(idRankedTables, idQueryGoal):
    
    accuracy = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            accuracy = 1
            break;

    return accuracy

In [4]:
corpusTrain = []

for rowId, articleTitle, tableTitle, match in trainDataset:
    
    corpusTrain.append(str(articleTitle))
    corpusTrain.append(str(tableTitle))

In [162]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(corpusTrain)]
doc2VecModel = Doc2Vec(documents, vector_size=10, window=2, min_count=1, workers=4)

In [163]:
from gensim.test.utils import get_tmpfile

fname = get_tmpfile("my_doc2vec_model")

doc2VecModel.save(fname)

doc2VecModel = Doc2Vec.load(fname)

/home/lss9/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [164]:
x_train = []
y_train = []


for rowId, articleTitle, tableTitle, match in trainDataset:
    
    tableTitleWords = tknzr.tokenize(str(articleTitle))
    vector1 = doc2VecModel.infer_vector(tableTitleWords)
    
    tableTitleWords = tknzr.tokenize(str(tableTitle))
    vector2 = doc2VecModel.infer_vector(tableTitleWords)
    
    vectorFinal = np.concatenate((vector1, vector2), axis=0)
    
    x_train.append(vectorFinal)
         
    y_train.append(match)

In [165]:
x1_train = np.asarray(x_train, dtype=np.float32)
y1_train = np.asarray(y_train)

In [166]:
import tensorflow as tf
from tensorflow import keras

In [245]:
modelDeep = keras.Sequential([
    keras.layers.Dense(20, input_dim=20),
    keras.layers.Dense(1024, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [246]:
modelDeep.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [399]:
modelDeep.fit(x1_train, y1_train, epochs=100, batch_size=32)

Train on 112519 samples
Epoch 1/100
112519/112519 [==============================] - 29s 258us/sample - loss: 0.5428 - accuracy: 0.7092
Epoch 2/100
112519/112519 [==============================] - 29s 255us/sample - loss: 0.5320 - accuracy: 0.7174
Epoch 3/100
112519/112519 [==============================] - 30s 269us/sample - loss: 0.5308 - accuracy: 0.7172
Epoch 4/100
112519/112519 [==============================] - 29s 258us/sample - loss: 0.5275 - accuracy: 0.7206
Epoch 5/100
112519/112519 [==============================] - 30s 265us/sample - loss: 0.5262 - accuracy: 0.7227
Epoch 6/100
112519/112519 [==============================] - 30s 264us/sample - loss: 0.5253 - accuracy: 0.7243
Epoch 7/100
112519/112519 [==============================] - 29s 260us/sample - loss: 0.5227 - accuracy: 0.7247
Epoch 8/100
112519/112519 [==============================] - 31s 272us/sample - loss: 0.5217 - accuracy: 0.7255
Epoch 9/100
112519/112519 [==============================] - 30s 265us/sample - 

KeyboardInterrupt: 

In [250]:
x_test = []
y_test = []

for rowId, articleTitle, tableTitle, match in validationDataset:
    
    tableTitleWords = tknzr.tokenize(str(articleTitle))
    vector1 = doc2VecModel.infer_vector(tableTitleWords)
    
    tableTitleWords = tknzr.tokenize(str(tableTitle))
    vector2 = doc2VecModel.infer_vector(tableTitleWords)
    
    vectorFinal = np.concatenate((vector1, vector2), axis=0)
    
    x_test.append(vectorFinal)
    y_test.append(match)

In [254]:
x1_test = np.asarray(x_test, dtype=np.float32)
y1_test = np.asarray(y_test)

In [255]:
y_predict = modelDeep.predict_classes(x1_test)

In [256]:
accuracy_score(y1_test, y_predict)

0.7035304767652384

In [384]:
def searchIndexing(articleTitle):
    
    result= indexingTables.search(
        index="tables", 
        body = {
        "_source": ["tablePgID","tablePgTitle"],
        "from" : 0,
        "size" : 100,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    articleTitle, 
              "fields": ["tablePgTitle"] 
            }
        }
    })
    
    return result

In [385]:
x_test = []
y_test = []



rowId, articleTitle, tableTitle, match = testDataset[2]
    
articleTitleWords = tknzr.tokenize(str(articleTitle))
vector1 = doc2VecModel.infer_vector(tableTitleWords)
    
result = searchIndexing(articleTitle)
    
rankedIDs = []
    
for hit in result['hits']['hits']:
    
    tablePgId = hit['_source']['tablePgID']
    tablePgTitle = hit['_source']['tablePgTitle']
        
    tableTitleWords = tknzr.tokenize(str(tablePgTitle))
    vector2 = doc2VecModel.infer_vector(tableTitleWords)
    
    vectorFinal = np.concatenate((vector1, vector2), axis=0)
        
    x_test.append(vectorFinal)
        
    rankedIDs.append(tablePgId)      

In [398]:
rowId

21230900

In [386]:
articleTitle

'french film queens deneuve adjani get along cannes chicago tribune'

In [388]:
x1_test = np.asarray(x_test, dtype=np.float32)

In [389]:
y_predict = modelDeep.predict(x1_test)

In [396]:
final = []

for i in range(len(y_predict)):
    
    final.append([rankedIDs[i],y_predict[i]])
    

In [397]:
final

[[38877647, array([0.19485548], dtype=float32)],
 [10798116, array([0.68189687], dtype=float32)],
 [30268846, array([0.68189687], dtype=float32)],
 [30268846, array([0.68189687], dtype=float32)],
 [30268846, array([0.68189687], dtype=float32)],
 [34023145, array([0.68189687], dtype=float32)],
 [34023145, array([0.68189687], dtype=float32)],
 [23384521, array([0.68189687], dtype=float32)],
 [10798116, array([0.68189687], dtype=float32)],
 [37917993, array([0.68189687], dtype=float32)],
 [37917993, array([0.68189687], dtype=float32)],
 [37917993, array([0.68189687], dtype=float32)],
 [30268846, array([0.68189687], dtype=float32)],
 [34023145, array([0.68189687], dtype=float32)],
 [16972176, array([0.68189687], dtype=float32)],
 [21231113, array([0.68189687], dtype=float32)],
 [45579, array([0.60516596], dtype=float32)],
 [10798116, array([0.68189687], dtype=float32)],
 [23384521, array([0.68189687], dtype=float32)],
 [21309330, array([0.68189687], dtype=float32)],
 [30268846, array([0.68

In [ ]:
def siamese_model(input_shape):
    
    left = tf.keras.Input(input_shape)
    right = tf.keras.Input(input_shape)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1024, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dense(512, activation = tf.nn.relu))
    model.add(tf.keras.layers.Dense(256, activation = tf.nn.sigmoid))
    
    left_encoded = model(left)
    right_encoded = model(right)
    
    L1_layer = tf.keras.layers.Lambda(lambda tensors:tf.keras.backend.abs(tensors[0] - tensors[1]))
    
    L1_distance = L1_layer([left_encoded, right_encoded])
    
    prediction = tf.keras.layers.Dense(1,activation='sigmoid')(L1_distance)
    
    siamese_net = tf.keras.Model(inputs=[left,right],outputs=prediction)
    
    return siamese_net

In [ ]:
left_train = np.array(left_train)
rigth_train = np.array(rigth_train)
label = np.array(label)

In [ ]:
model = siamese_model(10)

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="adam", metrics=['accuracy'])

In [ ]:
model.fit([left_train, rigth_train], label, epochs=100, batch_size=32)